In [1]:
from InstructorEmbedding import INSTRUCTOR


/home/ubuntu/miniconda3/envs/nvembed/lib/python3.9/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.embeddings.ollama import OllamaEmbeddings

In [2]:
df = pd.read_parquet("hf://datasets/m-ric/huggingface_doc_qa_eval/data/train-00000-of-00001.parquet")

/home/ubuntu/miniconda3/envs/nvembed/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df

,context,question,answer,source_doc,standalone_score,standalone_eval,relatedness_score,relatedness_eval,relevance_score,relevance_eval
0,`tokenizers-linux-x64-musl`\n\nThis is the **...,What architecture is the `tokenizers-linux-x64...,x86_64-unknown-linux-musl,huggingface/tokenizers/blob/main/bindings/node...,5,The question is asking about the specific arch...,5,The context directly specifies the architectur...,3,The question is asking for specific technical ...
1,!--Copyright 2023 The HuggingFace Team. All ri...,What is the purpose of the BLIP-Diffusion mode...,The BLIP-Diffusion model is designed for contr...,huggingface/diffusers/blob/main/docs/source/en...,5,The question is asking for the purpose of a sp...,5,The context provides a detailed description of...,3,The question asks about the purpose of the BLI...
2,Paper Pages\n\nPaper pages allow people to fi...,How can a user claim authorship of a paper on ...,By clicking their name on the corresponding Pa...,huggingface/hub-docs/blob/main/docs/hub/paper-...,5,The question is clear and does not depend on a...,5,The context provides a clear explanation of ho...,3,The question is specific to the Hugging Face H...
3,Datasets server API\n\n> API on 🤗 datasets\n\...,What is the purpose of the /healthcheck endpoi...,Ensure the app is running,huggingface/datasets-server/blob/main/services...,5,The question is asking for the purpose of a sp...,5,The context directly states the purpose of the...,4,"The question is specific and technical, asking..."
4,!--Copyright 2022 The HuggingFace Team. All ri...,What is the default context window size for Lo...,127 tokens,huggingface/transformers/blob/main/docs/source...,5,The question is asking for a specific paramete...,5,The context provides a specific detail about t...,3,"This question is specific and technical, askin..."
...,...,...,...,...,...,...,...,...,...,...
60,!--Copyright 2022 The HuggingFace Team. All ri...,What is the maximum size of a model checkpoint...,10GB,huggingface/transformers/blob/main/docs/source...,5,The question is asking for a specific piece of...,5,The context explicitly states that since versi...,3,"This question is specific and technical, focus..."
61,Gradio and W&B Integration\n\nRelated spaces:...,What is the purpose of Weights and Biases (W&B...,To track their machine learning experiments at...,gradio-app/gradio/blob/main/guides/06_integrat...,5,The question asks about the purpose of a speci...,5,The context provides a clear explanation of wh...,4,The question is asking about the purpose of We...
62,"--\ntitle: ""Intel and Hugging Face Partner to ...",What is the name of the open-source library cr...,Optimum,huggingface/blog/blob/main/intel.md,5,The question is asking for the name of a speci...,5,The context provided includes a detailed expla...,3,The question asks for the name of a specific o...
63,控制布局 (Controlling Layout)\n\n默认情况下，块中的组件是垂直排列...,What parameter is used to ensure that elements...,equal_height,gradio-app/gradio/blob/main/guides/cn/03_build...,5,The question is clear and does not rely on a s...,5,The context provides a clear explanation of ho...,3,The question is specific to the Gradio library...


In [4]:
len(df["context"].unique())

65

In [5]:
embeddings_model = OllamaEmbeddings(model="mxbai-embed-large")

In [6]:
embeddings_model.embed_documents(["hi"])

[[0.8039813041687012,
  0.22213809192180634,
  0.17949312925338745,
  0.6160699129104614,
  -0.400476336479187,
  -0.645444929599762,
  0.5577682852745056,
  0.28410470485687256,
  0.6385135054588318,
  0.5383370518684387,
  -0.08393163979053497,
  0.23798106610774994,
  -0.6613010168075562,
  0.3315821886062622,
  -0.8298481702804565,
  -0.6467488408088684,
  -0.4813886880874634,
  -0.3465003967285156,
  -0.07325087487697601,
  0.4417816996574402,
  -0.3764148950576782,
  0.39848315715789795,
  -1.217586636543274,
  -0.001576397567987442,
  -0.2441992610692978,
  0.4142182469367981,
  -0.15116235613822937,
  0.10708984732627869,
  0.7208569049835205,
  0.7969604730606079,
  0.03927374631166458,
  1.0480643510818481,
  0.2014443278312683,
  -0.6022903919219971,
  -0.037865445017814636,
  -0.7690998911857605,
  0.8191965222358704,
  -0.7320609092712402,
  -0.36952540278434753,
  -0.23632726073265076,
  0.06157906353473663,
  0.218727707862854,
  1.401124358177185,
  -0.5500829815864563,

In [7]:
def create_document_embeddings(dataframe):
    # context_embeddings = []
    document_ids = []
    contexts = []
    for index, row in dataframe.iterrows():
        contexts.append(row['context'])
        document_ids.append(row['source_doc'])
        
    return embeddings_model.embed_documents(contexts), document_ids

In [8]:
context_embeddings, document_ids = create_document_embeddings(df)

In [21]:
def retrieve_top_document(question, context_embeddings, document_ids):
    # Generate embedding for the question
    question_embedding = embeddings_model.embed_query(f'Represent this sentence for searching relevant passages: {question}')
    
    # Calculate cosine similarity between the question and all contexts
    similarities = cosine_similarity([question_embedding], context_embeddings)[0]
    
    # Get the index of the most similar context
    top_index = np.argmax(similarities)
    return document_ids[top_index]

# Step 5: Calculate k=1 Accuracy
def calculate_accuracy(dataframe, context_embeddings, document_ids):
    correct_count = 0
    total = len(dataframe)
    
    for index, row in dataframe.iterrows():
        question = row['question']
        actual_doc_id = row['source_doc']
        
        # Retrieve top matching document ID
        retrieved_doc_id = retrieve_top_document(question, context_embeddings, document_ids)
        
        # Check if the retrieved document ID matches the actual document ID
        if retrieved_doc_id == actual_doc_id:
            correct_count += 1
        print(question)
        print(actual_doc_id)
        print(retrieved_doc_id)
        print(correct_count)
    # Calculate accuracy
    accuracy = correct_count / total
    return accuracy

In [22]:
# Step 6: Run the accuracy calculation
accuracy = calculate_accuracy(df, context_embeddings, document_ids)
print(f"Retrieval k=1 Accuracy: {accuracy * 100:.2f}%")

What architecture is the `tokenizers-linux-x64-musl` binary designed for?

huggingface/tokenizers/blob/main/bindings/node/npm/linux-x64-musl/README.md
huggingface/tokenizers/blob/main/bindings/node/npm/linux-x64-musl/README.md
1
What is the purpose of the BLIP-Diffusion model?

huggingface/diffusers/blob/main/docs/source/en/api/pipelines/blip_diffusion.md
gradio-app/gradio/blob/main/demo/stable-diffusion/DESCRIPTION.md
1
How can a user claim authorship of a paper on the Hugging Face Hub?

huggingface/hub-docs/blob/main/docs/hub/paper-pages.md
huggingface/hub-docs/blob/main/docs/hub/paper-pages.md
2
What is the purpose of the /healthcheck endpoint in the Datasets server API?

huggingface/datasets-server/blob/main/services/api/README.md
huggingface/datasets-server/blob/main/services/api/README.md
3
What is the default context window size for Local Attention in the LongT5 model?

huggingface/transformers/blob/main/docs/source/en/model_doc/longt5.md
huggingface/blog/blob/main/nystromformer

In [5]:
from src.my_rag.components.embeddings.huggingface_embedding import HuggingFaceEmbedding

In [6]:
embedding_model = HuggingFaceEmbedding(
        model_name='dunzhang/stella_en_1.5B_v5',
#         load_in_8bit=True,
        trust_remote_code=True,
# max_memory={
#         0: '22000MB',
#         'cpu': '22000MB'
#     }
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
import torch
import gc

In [8]:
torch.cuda.empty_cache()
gc.collect()

79

In [9]:
def create_document_embeddings(dataframe):
    document_ids = []
    contexts = []
    for index, row in dataframe.iterrows():
        contexts.append(row['context'])
        document_ids.append(row['source_doc'])
        
    return embedding_model.embed(contexts, batch_size=1, instruction=""), document_ids

In [10]:
context_embeddings, document_ids = create_document_embeddings(df)

In [11]:
def retrieve_top_document(question, context_embeddings, document_ids):
    # Generate embedding for the question
    query_prefix = "Instruct: Given a question, retrieve most relavant passages that contains answer to the question \nQuery: "

    question_embedding = embedding_model.embed([question], instruction=query_prefix)
    
    # Calculate cosine similarity between the question and all contexts
    similarities = cosine_similarity(question_embedding, context_embeddings)[0]
    
    # Get the index of the most similar context
    top_index = np.argmax(similarities)
    return document_ids[top_index]

# Step 5: Calculate k=1 Accuracy
def calculate_accuracy(dataframe, context_embeddings, document_ids):
    correct_count = 0
    incorrect_count = 0
    total = len(dataframe)
    
    for index, row in dataframe.iterrows():
        question = row['question']
        actual_doc_id = row['source_doc']
        
        # Retrieve top matching document ID
        retrieved_doc_id = retrieve_top_document(question, context_embeddings, document_ids)
        
        # Check if the retrieved document ID matches the actual document ID
        if retrieved_doc_id == actual_doc_id:
            correct_count += 1
        else:
            incorrect_count += 1
            print(question)
            print(actual_doc_id)
            print(retrieved_doc_id)
            print(correct_count)
            print(incorrect_count)
    # Calculate accuracy
    print(total)
    print(correct_count)
    print(incorrect_count)
    accuracy = correct_count / total
    return accuracy

In [12]:
# Step 6: Run the accuracy calculation
accuracy = calculate_accuracy(df, context_embeddings, document_ids)
print(f"Retrieval k=1 Accuracy: {accuracy * 100:.2f}%")

What is the name of the large multimodal model that can solve image-text tasks and is based on Flamingo?

huggingface/transformers/blob/main/docs/source/en/tasks/idefics.md
huggingface/transformers/blob/main/docs/source/en/tasks_explained.md
8
1
What is the dimension of the feature vector for the base BERT model?

huggingface/course/blob/main/subtitles/en/raw/chapter1/05_encoders.md
huggingface/transformers/blob/main/docs/source/en/model_doc/albert.md
35
2
What is the purpose of tokenizers in the NLP pipeline?

huggingface/course/blob/main/chapters/en/chapter2/4.mdx
huggingface/tokenizers/blob/main/docs/source-doc-builder/components.mdx
42
3
What is the purpose of Weights and Biases (W&B) for data scientists and machine learning scientists?

gradio-app/gradio/blob/main/guides/06_integrating-other-frameworks/Gradio-and-Wandb-Integration.md
huggingface/course/blob/main/chapters/en/chapter1/10.mdx
58
4
What is the name of the open-source library created by Hugging Face to simplify Transfo

In [5]:
model = INSTRUCTOR('hkunlp/instructor-large')

/home/ubuntu/miniconda3/envs/nvembed/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


load INSTRUCTOR_Transformer
max_seq_length  512


/home/ubuntu/miniconda3/envs/nvembed/lib/python3.9/site-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dic

In [6]:
# Step 3: Create embeddings for contexts (documents) and build a database
def create_document_embeddings(dataframe):
    context_embeddings = []
    document_ids = []
    instruction = "Represent the github repository markdown document content for retrieval:"
    
    for index, row in dataframe.iterrows():
        context = row['context']
        embedding = model.encode([[instruction, context]])[0]  # Generate embedding for context
        context_embeddings.append(embedding)
        document_ids.append(row['source_doc'])
        
    return np.array(context_embeddings), document_ids

# Create embeddings for each context
context_embeddings, document_ids = create_document_embeddings(df)

In [7]:

# Step 4: Embed each question and perform retrieval
def retrieve_top_document(question, context_embeddings, document_ids):
    # Custom instruction for questions
    instruction = "Represent the github markdown-related question for retrieving the relevant document:"
    
    # Generate embedding for the question
    question_embedding = model.encode([[instruction, question]])[0]
    
    # Calculate cosine similarity between the question and all contexts
    similarities = cosine_similarity([question_embedding], context_embeddings)[0]
    
    # Get the index of the most similar context
    top_index = np.argmax(similarities)
    return document_ids[top_index]

# Step 5: Calculate k=1 Accuracy
def calculate_accuracy(dataframe, context_embeddings, document_ids):
    correct_count = 0
    total = len(dataframe)
    
    for index, row in dataframe.iterrows():
        question = row['question']
        actual_doc_id = row['source_doc']
        
        # Retrieve top matching document ID
        retrieved_doc_id = retrieve_top_document(question, context_embeddings, document_ids)
        
        # Check if the retrieved document ID matches the actual document ID
        if retrieved_doc_id == actual_doc_id:
            correct_count += 1
    
    # Calculate accuracy
    accuracy = correct_count / total
    return accuracy


In [8]:
# Step 6: Run the accuracy calculation
accuracy = calculate_accuracy(df, context_embeddings, document_ids)
print(f"Retrieval k=1 Accuracy: {accuracy * 100:.2f}%")

Retrieval k=1 Accuracy: 89.23%
